Intcode uitbreiding:
- Opcode 3 takes a single integer as input and saves it to the address given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
- Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50.
- Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
- Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
- Opcode 7 is less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
- Opcode 8 is equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

Parametre mode:
- parameter mode 0, position mode
- mode 1, immediate mode.



ABCDE
 1002

DE - two-digit opcode,      02 == opcode 2
 C - mode of 1st parameter,  0 == position mode       -  input1
 B - mode of 2nd parameter,  1 == immediate mode      -  input2
 A - mode of 3rd parameter,  0 == position mode,      -  input3
                                  omitted due to being a leading zero

In [2]:
def parse_code(code):
    A=int(code/10000)
    B=int((code-A*10000)/1000)
    C=int((code-A*10000-B*1000)/100)
    opcode=int((code-A*10000-B*1000-C*100))
    return(A,B,C,opcode)

def read_ab(ic,i,B,C):
    a=ic[i+1] if C==1 else ic[ic[i+1]]
    b=ic[i+2] if B==1 else ic[ic[i+2]]
    return (a,b)

def Incode(ic,inn):
    i=0
    output=[]
    while i<len(ic):
        if ic[i]==99:
            break
        else:
            (A,B,C,opcode)=parse_code(ic[i])
            #print(f'opcode:{opcode},ic[i]:{ic[i]}')
            if opcode==1:
                (a,b)=read_ab(ic,i,B,C)
                if A:
                    ic[i+3]=a+b
                else:
                    ic[ic[i+3]]=a+b
                #print('a:{},b:{},a+b:{}'.format(a,b,a+b))
            elif opcode==2:
                (a,b)=read_ab(ic,i,B,C)
                if A:
                    ic[i+3]=a*b
                else:
                    ic[ic[i+3]]=a*b
                #print('a:{},b:{},a*b:{}'.format(a,b,a*b))
            elif opcode==3:
                if C:
                    ic[i+1]=inn
                else:
                    ic[ic[i+1]]=inn
                #print(f'in:{inn}, immode:{immode1,immode2,immode3},imic:{ic[i+1]},ic:{ic[ic[i+1]]}')
            elif opcode==4:
                if C:
                    output.append(ic[i+1])
                else:
                    output.append(ic[ic[i+1]])
                #print(f'out:{output}, immode:{immode1,immode2,immode3},imic:{ic[i+1]},ic:{ic[ic[i+1]]}')
            elif opcode==5:
                #if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
                (a,b)=read_ab(ic,i,B,C)
                if a!=0:
                    i=b
                else:
                    i+=3
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            elif opcode==6:
                #if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.
                (a,b)=read_ab(ic,i,B,C)
                if a==0:
                    i=b
                else:
                    i+=3
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            elif opcode==7:
                #if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
                (a,b)=read_ab(ic,i,B,C)
                if a<b:
                    if A:
                        ic[i+3]=1
                    else:
                        ic[ic[i+3]]=1
                else: 
                    if A:
                        ic[i+3]=0
                    else:
                        ic[ic[i+3]]=0
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            elif opcode==8:
                #if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
                (a,b)=read_ab(ic,i,B,C)
                if a==b:
                    if A==1:
                        ic[i+3]=1
                    else:
                        ic[ic[i+3]]=1
                else: 
                    if A==1:
                        ic[i+3]=0
                    else:
                        ic[ic[i+3]]=0  
                #print(f'a:{a}b:{b},C:{C},B:{B}')
            else:
                print('error')
        if opcode in (1,2,7,8):
            i+=4
        elif opcode in (3,4):
            i+=2 
        
        #print(i)
        #print(output)
        #print('-----------------')
    return (output,ic)

In [3]:
#input:
ic_org=[3,225,1,225,6,6,1100,1,238,225,104,0,1102,35,92,225,1101,25,55,225,1102,47,36,225,1102,17,35,225,1,165,18,224,1001,224,-106,224,4,224,102,8,223,223,1001,224,3,224,1,223,224,223,1101,68,23,224,101,-91,224,224,4,224,102,8,223,223,101,1,224,224,1,223,224,223,2,217,13,224,1001,224,-1890,224,4,224,102,8,223,223,1001,224,6,224,1,224,223,223,1102,69,77,224,1001,224,-5313,224,4,224,1002,223,8,223,101,2,224,224,1,224,223,223,102,50,22,224,101,-1800,224,224,4,224,1002,223,8,223,1001,224,5,224,1,224,223,223,1102,89,32,225,1001,26,60,224,1001,224,-95,224,4,224,102,8,223,223,101,2,224,224,1,223,224,223,1102,51,79,225,1102,65,30,225,1002,170,86,224,101,-2580,224,224,4,224,102,8,223,223,1001,224,6,224,1,223,224,223,101,39,139,224,1001,224,-128,224,4,224,102,8,223,223,101,3,224,224,1,223,224,223,1102,54,93,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,1008,677,677,224,1002,223,2,223,1005,224,329,101,1,223,223,7,677,677,224,102,2,223,223,1006,224,344,101,1,223,223,108,677,677,224,1002,223,2,223,1006,224,359,1001,223,1,223,7,677,226,224,1002,223,2,223,1005,224,374,1001,223,1,223,1107,677,226,224,1002,223,2,223,1005,224,389,1001,223,1,223,107,226,677,224,102,2,223,223,1005,224,404,1001,223,1,223,1108,226,677,224,1002,223,2,223,1006,224,419,101,1,223,223,107,226,226,224,102,2,223,223,1005,224,434,1001,223,1,223,108,677,226,224,1002,223,2,223,1006,224,449,101,1,223,223,108,226,226,224,102,2,223,223,1006,224,464,1001,223,1,223,1007,226,226,224,1002,223,2,223,1005,224,479,101,1,223,223,8,677,226,224,1002,223,2,223,1006,224,494,101,1,223,223,1007,226,677,224,102,2,223,223,1006,224,509,101,1,223,223,7,226,677,224,1002,223,2,223,1005,224,524,101,1,223,223,107,677,677,224,102,2,223,223,1005,224,539,101,1,223,223,1008,677,226,224,1002,223,2,223,1005,224,554,1001,223,1,223,1008,226,226,224,1002,223,2,223,1006,224,569,1001,223,1,223,1108,226,226,224,102,2,223,223,1005,224,584,101,1,223,223,1107,226,677,224,1002,223,2,223,1005,224,599,1001,223,1,223,8,226,677,224,1002,223,2,223,1006,224,614,1001,223,1,223,1108,677,226,224,102,2,223,223,1005,224,629,1001,223,1,223,8,226,226,224,1002,223,2,223,1005,224,644,1001,223,1,223,1107,677,677,224,1002,223,2,223,1005,224,659,1001,223,1,223,1007,677,677,224,1002,223,2,223,1005,224,674,101,1,223,223,4,223,99,226]
ic_ex1=[3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9]
ic_ex2=[3,3,1105,-1,9,1101,0,0,12,4,12,99,1]
ic_ex3=[3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]

In [4]:
(output,ic)=Incode(ic_org,5)

In [5]:
print(output)

[9217546]


In [6]:
Z=1
if Z:
    print('dit werkt')
else:
    print('echt niet')

dit werkt
